In [5]:
alpha_name = "AlphaNetV1_Shift_2_Sequence_15_10d_ret_standscalar"
start_date = '2019-01-01'
end_date = '2021-06-01'
change = 5

In [6]:
import time
from Research.backtest.bt import BTDaily
from Research.feature.ft import FeatureAnalysis
import Research.utils.namespace as namespace
import Research.utils.normalization as norm
import numpy as np
import pandas as pd
import DataAPI
import os
input_path = r'/home/wuwenjun/factor_lib/ti0/wuwenjun/' + alpha_name
output_path = r'/home/wuwenjun/factor_lib/ti0/wuwenjun/'

alpha_list = []
trading_list = DataAPI.get_trading_days(start_date, end_date)
count = 0
for day in trading_list:
    file = os.path.join(input_path, str(day.year), day.strftime('%Y%m%d') + '.csv')
    try:
        df = pd.read_csv(file, dtype={0: str})
    except FileNotFoundError:
        count += 1
        continue
    flag = count % change
    path_output = os.path.join(output_path, str(alpha_name + "_" + str(flag)), str(day.year), day.strftime('%Y%m%d') + '.csv')
    alpha_list.append(str(alpha_name + "_" + str(flag)))
    if not os.path.exists(path_output):
        os.makedirs(os.path.dirname(path_output), exist_ok=True)
    df.to_csv(path_output, index=False, header=False)
    count += 1

In [7]:
pd.set_option('expand_frame_repr', False)
config_path = r'/home/ShareFolder/lgc/Modules/Research/config/feature_bt_template'
print('Loading the configuration from ' + config_path)
configs = namespace.load_namespace(config_path)
FT = FeatureAnalysis(configs, feature_path=r"/home/wuwenjun/factor_lib")
alpha_list = list(set(alpha_list))
print(alpha_list)
FT.load_feature_from_file(alpha_list, start_date, end_date, universe='Investable',
                                       timedelta=None, transformer=norm.standard_scale)
FT.load_return_data()

writing_path = r'/home/wuwenjun/Desktop/%s.txt' % alpha_name
f = open (writing_path,'w')
print("\n",file=f)
f.close()
for alpha_name in alpha_list:
    FT.get_intersection_ic(feature_name=alpha_name, truncate_fold=None, method='spearman',
                                     period=('1d', '3d', '5d'))
    ic_flag, trading_direction = FT.test_ic(alpha_name, verbose=False)
    df, df_all = FT.get_ic_summary_by_month(num=6)
    
    if trading_direction == -1:
        negative = True
    else:
        negative = False
    f = open (writing_path,'a')
    print("*"*100,end="\n"*3,file=f)
    print(alpha_name,end = "\n"*3,file=f)
    print(df,end="\n"*3,file = f)
    a = FT.get_top_return(alpha_name, negative= False, trade_type='long-only', transmission_rate=0.00025,
                            tax_rate=0.001, verbose=True,bt_price = "vwap",trade_period=6)
    print(a[1],end = "\n"*3+"*"*100 ,file = f)
    f.close()

Loading the configuration from /home/ShareFolder/lgc/Modules/Research/config/feature_bt_template


Loading Feature...:   4%|███▉                                                                                     | 131/2920 [00:00<00:04, 628.65it/s]

['AlphaNetV1_Shift_2_Sequence_15_10d_ret_standscalar_3', 'AlphaNetV1_Shift_2_Sequence_15_10d_ret_standscalar_0', 'AlphaNetV1_Shift_2_Sequence_15_10d_ret_standscalar_4', 'AlphaNetV1_Shift_2_Sequence_15_10d_ret_standscalar_1', 'AlphaNetV1_Shift_2_Sequence_15_10d_ret_standscalar_2']


Calculating IC value...: 100%|█████████████████████████████████████████████████████████████████████████████████████| 584/584 [00:00<00:00, 805.22it/s]



IC table for feature "AlphaNetV1_Shift_2_Sequence_15_10d_ret_standscalar_3" (mean std IR):
+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|      period       |  IC_AlphaNetV1_1d  |  IC_AlphaNetV1_2d  |  IC_AlphaNetV1_3d  |  IC_AlphaNetV1_4d  |  IC_AlphaNetV1_5d  |
+===================+====================+====================+====================+====================+====================+
| 20190107-20190630 |   3.80 4.82 0.79   |   5.37 5.19 1.04   |   5.36 6.44 0.83   |   5.25 6.34 0.83   |   6.13 5.77 1.06   |
+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| 20190701-20191231 |   1.04 4.22 0.25   |   2.15 4.35 0.49   |   2.44 4.42 0.55   |   3.19 4.48 0.71   |   3.76 4.76 0.79   |
+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| 20200101-20200630

...


AlphaNetV1_Shift_2_Sequence_15_10d_ret_standscalar_1 back-test result for long-only mode:

Back test summary: "long-only" mode
+-------------------+-------------+----------+-----------+------------+----------+-------------+-------------------------+-------------------+------------+
|      period       |  stock_num  |  return  |  ret_std  |  ret_year  |  sharpe  |  win_ratio  |      max_dd|period      |  max_dd_day|date  |  turnover  |
+===================+=============+==========+===========+============+==========+=============+=========================+===================+============+
| 20190102-20190630 |     211     |   3.00   |   5.85    |    6.23    |   1.10   |    54.70    | 4.04(20190116-20190408) |  0.96(20190131)   |   14.45    |
+-------------------+-------------+----------+-----------+------------+----------+-------------+-------------------------+-------------------+------------+
| 20190701-20191231 |     214     |   1.97   |   4.40    |    3.81    |   0.89   |    53.60 

Calculating IC value...: 100%|█████████████████████████████████████████████████████████████████████████████████████| 584/584 [00:00<00:00, 691.11it/s]



IC table for feature "AlphaNetV1_Shift_2_Sequence_15_10d_ret_standscalar_2" (mean std IR):
+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|      period       |  IC_AlphaNetV1_1d  |  IC_AlphaNetV1_2d  |  IC_AlphaNetV1_3d  |  IC_AlphaNetV1_4d  |  IC_AlphaNetV1_5d  |
+===================+====================+====================+====================+====================+====================+
| 20190104-20190630 |   3.06 6.14 0.50   |   4.24 6.36 0.67   |   4.97 5.81 0.85   |   5.34 6.36 0.84   |   5.85 6.83 0.86   |
+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| 20190701-20191231 |   3.33 5.03 0.66   |   3.05 4.77 0.64   |   2.84 4.79 0.59   |   3.43 4.21 0.81   |   3.65 4.04 0.90   |
+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| 20200101-20200630

Getting BT result for date: 20210531...: 100%|██████████████████████████████████████████████████████████████████████| 584/584 [00:05<00:00, 98.47it/s]


AlphaNetV1_Shift_2_Sequence_15_10d_ret_standscalar_2 back-test result for long-only mode:

Back test summary: "long-only" mode
+-------------------+-------------+----------+-----------+------------+----------+-------------+-------------------------+-------------------+------------+
|      period       |  stock_num  |  return  |  ret_std  |  ret_year  |  sharpe  |  win_ratio  |      max_dd|period      |  max_dd_day|date  |  turnover  |
+===================+=============+==========+===========+============+==========+=============+=========================+===================+============+
| 20190102-20190630 |     211     |   7.74   |   5.62    |   16.45    |   2.87   |    63.25    | 2.88(20190110-20190201) |  0.80(20190129)   |   14.57    |
+-------------------+-------------+----------+-----------+------------+----------+-------------+-------------------------+-------------------+------------+
| 20190701-20191231 |     214     |   0.07   |   4.46    |    0.14    |   0.03   |    50.40 